<a href="https://colab.research.google.com/github/biancamotac/Clima_bh/blob/main/Dados_Clim%C3%A1ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import pandas as pd
from datetime import datetime
from google.colab import drive
import plotly.express as px
import csv
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
api_key = "de566ecabca58322bf53863eb9f8f9f4"
cidade = "Belo Horizonte,BR"
url = f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={api_key}&units=metric&lang=pt_br"


response = requests.get(url)
response.raise_for_status()
dados = response.json()

dados_para_csv = {
    "data_hora": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "cidade": dados["name"],
    "temperatura": dados["main"]["temp"],
    "descricao": dados["weather"][0]["description"].capitalize(),
    "umidade": dados["main"]["humidity"],
    "vento": dados["wind"]["speed"]
}

print("Dados coletados com sucesso!")


Dados coletados com sucesso!


In [4]:
arquivo_csv = "/content/drive/MyDrive/clima_bh.csv"

try:
    with open(arquivo_csv, "x", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=dados_para_csv.keys())
        writer.writeheader()
        writer.writerow(dados_para_csv)
except FileExistsError:
    with open(arquivo_csv, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=dados_para_csv.keys())
        writer.writerow(dados_para_csv)

print(f"Dados salvos em {arquivo_csv}!")


Dados salvos em /content/drive/MyDrive/clima_bh.csv!


In [6]:
df = pd.read_csv(arquivo_csv)
df['data_hora'] = pd.to_datetime(df['data_hora'])

df['data'] = df['data_hora'].dt.date

df_diario = df.groupby('data').agg({
    'temperatura': ['mean', 'max', 'min'],
    'umidade': 'mean',
    'vento': 'mean'
}).reset_index()

df_diario.columns = ['data', 'temp_media', 'temp_max', 'temp_min', 'umidade_media', 'vento_medio']
df_diario.head()


,data,temp_media,temp_max,temp_min,umidade_media,vento_medio
0,2025-09-30,26.28,26.28,26.28,34.0,3.51


In [10]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_diario['data'], y=df_diario['temp_media'],
    name='Temperatura Média (°C)',
    marker_color='#ffe4e1'
))

fig.add_trace(go.Bar(
    x=df_diario['data'], y=df_diario['umidade_media'],
    name='Umidade Média (%)',
    marker_color='#fdf9c4'
))

fig.add_trace(go.Bar(
    x=df_diario['data'], y=df_diario['vento_medio'],
    name='Vento Médio (m/s)',
    marker_color='#d0bdf6'
))

fig.update_layout(
    barmode='group',
    title='Clima Diário em Belo Horizonte',
    xaxis_title='Data',
    yaxis_title='Valores',
    xaxis_tickangle=-45,
    template='plotly'
)

fig.show()